$$
\def\CC{\bf C}
\def\QQ{\bf Q}
\def\RR{\bf R}
\def\ZZ{\bf Z}
\def\NN{\bf N}
$$
# Getting Started

## Import pyopenms

After installation, you should be able to import pyopenms as a package

In [ ]:
import pyopenms

which should now give you access to all of pyopenms. You should now be
able to interact with the OpenMS library and, for example, read and
write mzML files:

In [ ]:
from pyopenms import *
exp = MSExperiment()
MzMLFile().store("testfile.mzML", exp)

which will create an empty mzML file called $testfile.mzML$.

## Getting help

There are multiple ways to get information about the available functions
and methods. We can inspect individual pyOpenMS objects through the
`help` function:

In [ ]:
from pyopenms import *
help(MSExperiment)

In [ ]:
class MSExperiment(builtins.object)
 |  Cython implementation of _MSExperiment
 |   -- Inherits from ['ExperimentalSettings', 'RangeManager2']
 |  
 |  In-Memory representation of a mass spectrometry experiment.
 |  -----
 |  Contains the data and metadata of an experiment performed with an MS (or
 |  HPLC and MS). This representation of an MS experiment is organized as list
 |  of spectra and chromatograms and provides an in-memory representation of
 |  popular mass-spectrometric file formats such as mzXML or mzML. The
 |  meta-data associated with an experiment is contained in
 |  ExperimentalSettings (by inheritance) while the raw data (as well as
 |  spectra and chromatogram level meta data) is stored in objects of type
 |  MSSpectrum and MSChromatogram, which are accessible through the getSpectrum
 |  and getChromatogram functions.
 |  -----
 |  Spectra can be accessed by direct iteration or by getSpectrum(),
 |  while chromatograms are accessed through getChromatogram().
 |  See help(ExperimentalSettings) for information about meta-data.
 |  
 |  Methods defined here:

 [...]

which lists information on the `pyopenms.MSExperiment` class, including
a description of the main purpose of the class and how the class is
intended to be used. Additional useful information is presented in the
`Inherits from` section which points to additional classes that act as
base classes to `pyopenms.MSExperiment` and that contain further
information. The list of available methods is long (but does *not*
include methods from the base classes) and reveals that the class
exposes methods such as `getNrSpectra()` and `getSpectrum(id)` where the
argument `id` indicates the spectrum identifer. The command also lists
the signature for each function, allowing users to identify the function
arguments and return types. We can gain further information about
exposed methods by investigating the documentation of the base classes:

In [ ]:
from pyopenms import *
help(ExperimentalSettings)

Help on class ExperimentalSettings in module pyopenms.pyopenms_4:

class ExperimentalSettings(builtins.object)
 |  Cython implementation of _ExperimentalSettings
 |   -- Inherits from ['DocumentIdentifier', 'MetaInfoInterface']
 |  
 |  Description of the experimental settings, provides meta-information
 |  about an LC-MS/MS injection.
 |  
 |  Methods defined here:

 [...]

We could now continue our investigation by reading the documentation of
the base classes `DocumentIdentifier` and `MetaInfoInterface`, but we
will leave this exercise for the interested reader. In order to get more
information about the wrapped functions, we can also consult the
[pyOpenMS manual](http://proteomics.ethz.ch/pyOpenMS_Manual.pdf) which
references to all wrapped functions. For a more complete documentation
of the underlying wrapped methods, please consult the official OpenMS
documentation, in this case the [MSExperiment
documentation](https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/classOpenMS_1_1MSExperiment.html).

## First look at data

### File reading

pyOpenMS supports a variety of different files through the
implementations in OpenMS. In order to read mass spectrometric data, we
can download the [mzML example
file](http://proteowizard.sourceforge.net/example_data/tiny.pwiz.1.1.mzML)

In [ ]:
from urllib.request import urlretrieve
# from urllib import urlretrieve  # use this code for Python 2.x
from pyopenms import *
urlretrieve ("http://proteowizard.sourceforge.net/example_data/tiny.pwiz.1.1.mzML", "tiny.pwiz.1.1.mzML")
exp = MSExperiment()
MzMLFile().load("tiny.pwiz.1.1.mzML", exp)

which will load the content of the "tiny.pwiz.1.1.mzML" file into the
`exp` variable of type `MSExperiment`. We can now inspect the properties
of this object:

In [ ]:
help(exp)

In [ ]:
class MSExperiment(builtins.object)
 |  Cython implementation of _MSExperiment
 |   -- Inherits from ['ExperimentalSettings', 'RangeManager2']


 [...]

 |  Methods defined here:

 [...]

 |  getNrChromatograms(...)
 |      Cython signature: size_t getNrChromatograms()
 |
 |  getNrSpectra(...)
 |      Cython signature: size_t getNrSpectra()
 |

 [...]

which indicates that the variable `exp` has (among others) the functions
`getNrSpectra` and `getNrChromatograms`. We can now try these functions:

In [ ]:
exp.getNrSpectra()

4

In [ ]:
exp.getNrChromatograms()

2

and indeed we see that we get information about the underlying MS data.
We can iterate through the spectra as follows:

### Iteration

In [ ]:
for spec in exp:
  print ("MS Level:", spec.getMSLevel())

MS Level: 1
MS Level: 2
MS Level: 1
MS Level: 1

This iterates through all available spectra, we can also access spectra
through the `[]` operator:

In [ ]:
print ("MS Level:", exp[1].getMSLevel())

MS Level: 2

Note that `spec[1]` will access the *second* spectrum (arrays start at
`0`). We can access the raw peaks through `get_peaks()` :

In [ ]:
spec = exp[1]
mz, intensity = spec.get_peaks()
sum(intensity)

110

Which will access the data using a numpy array, storing the *m/z*
information in the `mz` vector and the intensity in the `i` vector.
Alternatively, we can also iterate over individual peak objects as
follows (this tends to be slower):

In [ ]:
for peak in spec:
  print (peak.getIntensity())

20.0
18.0
16.0
14.0
12.0
10.0
8.0
6.0
4.0
2.0

### TIC calculation

Here, we will apply what we have learned to perform a common function in
mass spectrometry: calculating the total ion current. While some mzML
files already contain a pre-computed TIC, we will show you how to
calculate the TIC for MS1 and MS2 spectra separately (generally only MS1
TIC is provided).

With this information, we can write a function that calculates the total
ion current (TIC) for a given ms level:

In [ ]:
# Calculates total ion chromatogram of an LC-MS/MS experiment
def calcTIC(exp, mslevel):
    tic = 0
    # Iterate through all spectra of the experiment
    for spec in exp:
        # Only calculate TIC for matching (MS1) spectra
        if spec.getMSLevel() == mslevel:
            mz, i = spec.get_peaks()
            tic += sum(i)
    return tic

To calculate a TIC we would now call the function:

In [ ]:
calcTIC(exp, 1)

240.0

In [ ]:
sum([sum(s.get_peaks()[1]) for s in exp if s.getMSLevel() == 1])

240.0

In [ ]:
calcTIC(exp, 2)

110.0

Note how one can compute the same property using list comprehensions in
Python (see line number 3 in the above code which computes the TIC using
filtering properties of Python list comprehensions
(`s.getMSLevel() == 1`) and computes the sum over all peaks (right
`sum`) and the sum over all spectra (left `sum`) to retrieve the TIC).

One can access the retention times and intensities of the TIC in
different ways and generate a 2D graph using `matplotlib` :

In [ ]:
from pyopenms import *
import matplotlib.pyplot as plt
from urllib.request import urlretrieve

# retrieve MS data
gh = 'https://raw.githubusercontent.com/OpenMS/OpenMS/develop'
urlretrieve (gh+'/src/tests/topp/FeatureFinderMetaboIdent_1_input.mzML',
            'ms_data.mzML')

# load MS data into MSExperiment()
exp = MSExperiment()
MzMLFile().load("ms_data.mzML", exp)

# choose one of the following three methods to access the TIC data
# 1) get TIC data with exp.getTIC() function
tic = exp.getTIC()
retention_times, intensities = tic.get_peaks()

# 2) get TIC data using list comprehensions
retention_times = [spec.getRT() for spec in exp]
intensities = [sum(spec.get_peaks()[1]) for spec in exp if spec.getMSLevel() == 1]

# 3) get TIC data looping over spectra in MSExperiment()
retention_times = []
intensities = []
for spec in exp:
    if spec.getMSLevel() == 1:
        retention_times.append(spec.getRT())
        intensities.append(sum(spec.get_peaks()[1]))

# plot retention times and intensities and add labels
plt.plot(retention_times, intensities)

plt.title('TIC')
plt.xlabel('time (s)')
plt.ylabel('intensity (cps)')

plt.show()

![image](img/TICPlot.png)

[![Launch Binder](./img/launch_binder.jpg)](https://mybinder.org/v2/gh/OpenMS/pyopenms-extra/master+ipynb?urlpath=lab/tree/docs/source/getting_started.ipynb)